## Write dictionary into files

In [1]:
from pynq.pl import TCL
from pynq.pl import HWH

tcl_parser = TCL('/home/xilinx/pynq/overlays/base/base.tcl')
hwh_parser = HWH('/home/xilinx/pynq/overlays/base/base.hwh')

```python
import pickle

def dump_pickle(parser, dict_name):
    with open('tcl_{}.txt'.format(dict_name), 'wb') as f:
        pickle.dump(getattr(parser, dict_name), f)

def load_pickle(dict_name):
    with open('tcl_{}.txt'.format(dict_name), 'rb') as f:
        ret_dict = pickle.load(f)
    return ret_dict
```

## Test `ip_dict`

In [2]:
d = 'ip_dict'
golden = getattr(tcl_parser, d)
dut = getattr(hwh_parser, d)

for j in [golden, dut]:
    for i in ['iop_pmoda/mb', 'iop_pmodb/mb', 'iop_arduino/mb']:
        assert i not in j, \
            'Microblaze should not be in the ip dict.'

for i in golden:
    assert i in dut, \
        '{} in TCL ip_dict, not in HWH ip_dict.'.format(i)
    for key in ['phys_addr', 'addr_range', 'type', 'state', 'fullpath']:
        assert golden[i][key] == dut[i][key]

for i in dut:
    assert i in golden, \
        '{} in HWH ip_dict, not in TCL ip_dict.'.format(i)

## Test `gpio_dict`

In [3]:
d = 'gpio_dict'
golden = getattr(tcl_parser, d)
dut = getattr(hwh_parser, d)

for i in golden:
    assert i in dut, \
        '{} in TCL gpio_dict, not in HWH gpio_dict.'.format(i)
    for key in ['index', 'state']:
        assert golden[i][key] == dut[i][key]

# Some pins at a hierarchy block are not available in HWH parser
for i in dut:
    assert i in golden, \
        '{} in HWH gpio_dict, not in TCL gpio_dict.'.format(i)
    for j in dut[i]['pins']:
        assert j in golden[i]['pins'], \
            '{} in pins of HWH gpio_dict, not in TCL gpio_dict.'.format(j)

## Test `interrupt_pins`

In [4]:
d = 'interrupt_pins'
golden = getattr(tcl_parser, d)
dut = getattr(hwh_parser, d)

# The common entries must be the same
for i in golden:
    if i in dut:
        for key in ['controller', 'index', 'fullpath']:
            assert golden[i][key] == dut[i][key]

## Test `interrupt_controllers`

In [5]:
d = 'interrupt_controllers'
golden = getattr(tcl_parser, d)
dut = getattr(hwh_parser, d)
assert golden == dut

## Test `clock_dict`

In [8]:
d = 'clock_dict'
golden = getattr(tcl_parser, d)
dut = getattr(hwh_parser, d)
assert golden == dut